In [1]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")

## Helper Functions to reduce memory and check momory usage

 Since the data is pretty large, we can convert data type to save memory.

In [2]:
# Check memory usage
def memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2)

def size_fmt(num, suffix = "B"):
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, "Yi", suffix)

In [3]:
# Memory Reducer

def reduce_mem_usage(df, verbose = True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32) 
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
              .format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df    

In [4]:
# Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on = merge_on, how = 'left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

## Set Variables and data path

In [5]:
# Variables
Target = "sales"
Last_train = 1941
iden_index = ["id","d"] # columns that can be used as an identifier

In [7]:
# Datapath
DataPath = "/PATH/"

## Data Loading

In [41]:
print("Load Data")
train_df = pd.read_csv(DataPath + "/sales_train_evaluation.csv")
prices_df = pd.read_csv(DataPath + "/sell_prices.csv")
calendar_df = pd.read_csv(DataPath + "/calendar.csv")

Load Data


In [42]:
# Transform horizontal into vertical data to fit the model
# indexes: "id", "item_id", "dept_id", "cat_id", "store_id", "state_id"
# labels: "d_"
print("Create Grid")
idx_cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
grid_df = pd.melt(train_df, id_vars = idx_cols, var_name = 'd', value_name = Target)

print(f"horizontal train rows {len(train_df)}")
print(f"vertical train rows {len(train_df)}")

Create Grid
horizontal train rows 30490
vertical train rows 30490


In [43]:
# Add test data into grid in order to make predictions
add_grid = pd.DataFrame()
for i in range(1, 29):
    temp_df = train_df[idx_cols]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = "d_" + str(Last_train + i)
    temp_df[Target] = np.nan
    add_grid = pd.concat([add_grid, temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)

In [44]:
# Remove temporary dfs and original data
del temp_df, add_grid
del train_df

In [45]:
# Check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',size_fmt(grid_df.memory_usage(index=True).sum())))
# Free memory by converting "string" to categorical data
for col in idx_cols:
    grid_df[col] = grid_df[col].astype("category")
# Check memory usage now
print("{:>20}: {:>8}".format('Reduced grid_df',size_fmt(grid_df.memory_usage(index=True).sum())))

    Original grid_df:   3.6GiB
     Reduced grid_df:   1.3GiB


## First week that a product was in store

 A lot of 0 sales were observed in the dataset, and many of them are because 
 the products were not in store at that time. From the price table we are able
 to get this information.

In [47]:
# Discover the first week that a product was in store
print("Release week")
release_df = prices_df.groupby(["store_id", "item_id"])["wm_yr_wk"].agg(["min"]).reset_index()
release_df.columns = ["store_id", "item_id", "release"]

Release week


In [48]:
# Merge release info into grid_df
grid_df = merge_by_concat(grid_df, release_df, ["store_id", "item_id"])
del release_df

In [49]:
# Remove the rows if the week is earlier than release week from grid
grid_df = merge_by_concat(grid_df, calendar_df[["wm_yr_wk", "d"]], ["d"])
grid_df = grid_df[grid_df["wm_yr_wk"] >= grid_df["release"]]
grid_df = grid_df.reset_index(drop = True)

# Check memory usage
print("{:>20}: {:>8}".format('Original grid_df',size_fmt(grid_df.memory_usage(index=True).sum())))

grid_df["release"] = grid_df["release"] - grid_df["release"].min()
grid_df["release"] = grid_df["release"].astype(np.int16)

# Check memory usage after converting
print("{:>20}: {:>8}".format('Reduced grid_df',size_fmt(grid_df.memory_usage(index=True).sum())))

    Original grid_df:   1.8GiB
     Reduced grid_df:   1.5GiB


In [50]:
print("Save part1")
grid_df.to_pickle("M5_part_1.pkl")
print("Size:", grid_df.shape)

Save part1
Size: (47735397, 10)


## Load prices and feature engineering for prices

In [51]:
# Prices
print("Prices features")

# Basic aggregation: price min, max, mean, std
prices_df["price_min"] = prices_df.groupby(["store_id", "item_id"])["sell_price"].transform("min")
prices_df["price_max"] = prices_df.groupby(["store_id", "item_id"])["sell_price"].transform("max")
prices_df["price_mean"] = prices_df.groupby(["store_id", "item_id"])["sell_price"].transform("mean")
prices_df["price_std"] = prices_df.groupby(["store_id", "item_id"])["sell_price"].transform("std")

# Price normalization
prices_df["sell_price"] = prices_df["sell_price"] / prices_df["price_max"]

# Some prices are inflated and some are stable
prices_df["price_nunique"] = prices_df.groupby(["store_id", "item_id"])["sell_price"].transform("nunique")
prices_df["item_nunique"] = prices_df.groupby(["store_id", "sell_price"])["item_id"].transform("nunique")

Prices features


In [52]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [53]:
# Having month and year as windows and do some rolling aggregation
calendar_prices = calendar_df[["wm_yr_wk", "month", "year"]]
calendar_prices = calendar_prices.drop_duplicates(subset = ["wm_yr_wk"])
prices_df = prices_df.merge(calendar_prices, on = ["wm_yr_wk"], how = "left")

del calendar_prices

In [54]:
# Price shifted by week, month mean and year mean
prices_df['price_momentum'] = prices_df['sell_price'] / prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_m'] = prices_df['sell_price'] / prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price'] / prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

In [55]:
# Merge prices and save part 2
original_cols = list(grid_df)
grid_df = grid_df.merge(prices_df, on = ["store_id", "item_id", "wm_yr_wk"], how = "left")
keep_cols = [col for col in list(grid_df) if col not in original_cols]
grid_df = grid_df[iden_index + keep_cols]
grid_df = reduce_mem_usage(grid_df)

Mem. usage decreased to 1867.97 Mb (64.0% reduction)


In [56]:
# Save part2
grid_df.to_pickle('M5_part_2.pkl')
print('Size:', grid_df.shape)

del prices_df

Size: (47735397, 14)


In [62]:
grid_df = pd.read_pickle("M5_part_1.pkl")
grid_df = grid_df[iden_index]

## Load calendar and feature engineering for calendar

In [63]:
# Merge calendar
icols = ["date",
         "d",
         "event_name_1",
         "event_type_1",
         "event_name_2",
         "event_type_2",
         "snap_CA",
         "snap_TX",
         "snap_WI"]
grid_df = grid_df.merge(calendar_df[icols], on = "d", how = "left")

# Convert data types to save memory
icols = ["event_name_1",
         "event_type_1",
         "event_name_2",
         "event_type_2",
         "snap_CA",
         "snap_TX",
         "snap_WI"]

for col in icols:
    grid_df[col] = grid_df[col].astype("category")

In [64]:
# date and date features
grid_df["date"] = pd.to_datetime(grid_df["date"])

In [65]:
grid_df.head()

,id,d,date,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_008_CA_1_evaluation,d_1,2011-01-29,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_009_CA_1_evaluation,d_1,2011-01-29,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_010_CA_1_evaluation,d_1,2011-01-29,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_012_CA_1_evaluation,d_1,2011-01-29,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_015_CA_1_evaluation,d_1,2011-01-29,NaN,NaN,NaN,NaN,0,0,0


In [66]:
# Retrive day, week, month, year info from calendar data
grid_df["tm_d"] = grid_df["date"].dt.day.astype(np.int8)
grid_df["tm_w"] = grid_df["date"].dt.week.astype(np.int8)
grid_df["tm_m"] = grid_df["date"].dt.month.astype(np.int8)
grid_df["tm_y"] = grid_df["date"].dt.year
grid_df["tm_y"] = (grid_df["tm_y"] - grid_df["tm_y"].min()).astype(np.int8)
grid_df["tm_wm"] = grid_df["tm_d"].apply(lambda x: ceil(x/7)).astype(np.int8)

grid_df["tm_dw"] = grid_df["date"].dt.dayofweek.astype(np.int8)
grid_df["tm_w_end"] = (grid_df["tm_dw"] >= 5).astype(np.int8)

del grid_df["date"]

In [67]:
# Save part 3
print("Save part 3")
grid_df.to_pickle('M5_part_3.pkl')
print('Size:', grid_df.shape)

Save part 3
Size: (47735397, 16)


In [68]:
del calendar_df
del grid_df

In [69]:
# Additional cleaning
grid_df = pd.read_pickle('M5_part_1.pkl')
grid_df["d"] = grid_df["d"].apply(lambda x: x[2:]).astype(np.int16)

# Remove "wm_yr_wk"
del grid_df['wm_yr_wk']
grid_df.to_pickle('M5_part_1.pkl')

del grid_df

## Create lag features

In [70]:
# Lag features
grid_df = pd.read_pickle('M5_part_1.pkl')

# Need "id", "d", sales to make lags and rollings
grid_df = grid_df[["id", "d", "sales"]]
shift_day = 28

In [71]:
start_time = time.time()
print("Create lag features")

lag_days = [col for col in range(shift_day, shift_day + 15)]
grid_df = grid_df.assign(**{
        '{}_lag_{}'.format(col, l): grid_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in lag_days
        for col in [Target]
    })

# Save some memory
for col in list(grid_df):
    if 'lag' in col:
        grid_df[col] = grid_df[col].astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

Create lag features
6.41 min: Lags


## Create rolling features

In [72]:
# Rolling features
start_time = time.time()
print("Create rolling features")

for i in [7, 14, 28, 56, 182, 364]:
    print("Rolling period:" ,i)
    grid_df["rolling_mean_" + str(i)] = grid_df.groupby(["id"])[Target].transform(lambda x: x.shift(shift_day).rolling(i).mean()).astype(np.float16)
    grid_df["rolling_std_" + str(i)] = grid_df.groupby(["id"])[Target].transform(lambda x: x.shift(shift_day).rolling(i).std()).astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))    

Create rolling features
Rolling period: 7
Rolling period: 14
Rolling period: 28
Rolling period: 56
Rolling period: 182
Rolling period: 364
6.67 min: Lags


In [73]:
# Rolling with sliding shift
start_time = time.time()
for d_shift in [1, 7, 14]:
    print("Shift period:", d_shift)
    for d_window in [7, 14, 28, 56]:
        col_name = "rolling_mean_tmp_" + str(d_shift) + "_" + str(d_window)
        grid_df[col_name] = grid_df.groupby(["id"])[Target].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))        

Shift period: 1
Shift period: 7
Shift period: 14
6.75 min: Lags


In [74]:
# Emport the data
print("Save lags and rollings")
grid_df.to_pickle("lags_df_" + str(shift_day) + ".pkl")

Save lags and rollings


In [75]:
# Check all the lag features, rolling means and rolling means for sliding windows
grid_df.head()

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,...,rolling_mean_tmp_1_28,rolling_mean_tmp_1_56,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_28,rolling_mean_tmp_7_56,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_28,rolling_mean_tmp_14_56
0,HOBBIES_1_008_CA_1_evaluation,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_009_CA_1_evaluation,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_010_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_012_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_015_CA_1_evaluation,1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Mean encoding

In [76]:
# Mean encoding
grid_df = pd.read_pickle("M5_part_1.pkl")
grid_df[Target][grid_df["d"] > (Last_train - 28)] = np.nan
base_cols = list(grid_df)

mcols = [['state_id'],
        ['store_id'],
        ['cat_id'],
        ['dept_id'],
        ['state_id', 'cat_id'],
        ['state_id', 'dept_id'],
        ['store_id', 'cat_id'],
        ['store_id', 'dept_id'],
        ['item_id'],
        ['item_id', 'state_id'],
        ['item_id', 'store_id']]

for col in mcols:
    print("Encoding", col)
    col_name = "_" + "_".join(col) + "_"
    grid_df["enc" + col_name + "mean"] = grid_df.groupby(col)[Target].transform("mean").astype(np.float16)
    grid_df["enc" + col_name + "std"] = grid_df.groupby(col)[Target].transform("std").astype(np.float16)
    
keep_cols = [col for col in list(grid_df) if col not in base_cols]
grid_df = grid_df[["id", "d"] + keep_cols]

Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']


In [148]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46881677 entries, 0 to 46881676
Data columns (total 24 columns):
id                           category
d                            int16
enc_state_id_mean            float16
enc_state_id_std             float16
enc_store_id_mean            float16
enc_store_id_std             float16
enc_cat_id_mean              float16
enc_cat_id_std               float16
enc_dept_id_mean             float16
enc_dept_id_std              float16
enc_state_id_cat_id_mean     float16
enc_state_id_cat_id_std      float16
enc_state_id_dept_id_mean    float16
enc_state_id_dept_id_std     float16
enc_store_id_cat_id_mean     float16
enc_store_id_cat_id_std      float16
enc_store_id_dept_id_mean    float16
enc_store_id_dept_id_std     float16
enc_item_id_mean             float16
enc_item_id_std              float16
enc_item_id_state_id_mean    float16
enc_item_id_state_id_std     float16
enc_item_id_store_id_mean    float16
enc_item_id_store_id_std     float1

In [78]:
# Export the data
print('Save Mean/Std encoding')
grid_df.to_pickle('mean_encoding_df.pkl')

Save Mean/Std encoding


## Feature selection

In [217]:
mean_features = ["enc_cat_id_mean",
                "enc_cat_id_std",
                "enc_dept_id_mean",
                "enc_dept_id_std",
                "enc_item_id_mean",
                "enc_item_id_std"]

In [218]:
# Custom features and feature selection
# Read data
grid_df = pd.concat([pd.read_pickle("M5_part_1.pkl"),
                     pd.read_pickle("M5_part_2.pkl").iloc[:,2:],
                     pd.read_pickle("M5_part_3.pkl").iloc[:,2:],
                     pd.read_pickle("mean_encoding_df.pkl")[mean_features]],
                     axis = 1)

In [220]:
# Subsampling to make calculations faster
# Keep only 5% of the id
keep_id = np.array_split(list(grid_df["id"].unique()),20)[0]
grid_df = grid_df[grid_df["id"].isin(keep_id)].reset_index(drop = True)

In [221]:
# Baseline model
Seed = 42 # Random seed for everything
random.seed(Seed)
np.random.seed(Seed)
N_cores = psutil.cpu_count() # Avaliable CPU cores

# Drop the data from test part
grid_df = grid_df[grid_df["d"] <= Last_train].reset_index(drop = True)

# Features exclude from the training
remove_features = ["id","d",Target]

In [222]:
# Baseline model would help check new features performance
import lightgbm as lgb

lgb_params = {
                    'boosting_type': 'gbdt',         # Standart boosting type
                    'objective': 'regression',       # Standart loss for RMSE
                    'metric': ['rmse'],              # as we will use rmse as metric "proxy"
                    'subsample': 0.8,                
                    'subsample_freq': 1,
                    'learning_rate': 0.05,           # 0.5 is "fast enough" for us
                    'num_leaves': 2**7-1,            # We will need model only for fast check
                    'min_data_in_leaf': 2**8-1,      # So we want it to train faster even with drop in generalization 
                    'feature_fraction': 0.8,
                    'n_estimators': 5000,            # We don't want to limit training (you can change 5000 to any big enough number)
                    'early_stopping_rounds': 30,     # We will stop training almost immediately (if it stops improving) 
                    'seed': Seed,
                    'verbose': -1,
                } 

# RMSE
def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

def make_fast_test(df):
    features_cols = [col for col in list(grid_df) if col not in remove_features]
    # Use last 28 days data as validation data
    train_X = df[df.d <= (Last_train - 28)][features_cols]
    train_y = df[df.d <= (Last_train - 28)][Target]
    valid_X = df[df.d > (Last_train - 28)][features_cols]
    valid_y = df[df.d > (Last_train - 28)][Target]
    
    train_data = lgb.Dataset(train_X, label = train_y)
    valid_data = lgb.Dataset(valid_X, label = valid_y)
    
    estimator = lgb.train(
                        lgb_params,
                        train_data,
                        valid_sets = [train_data,valid_data],
                        verbose_eval = 500,
                    )
    
    return estimator

In [223]:
# Make baseline model
baseline_model = make_fast_test(grid_df)

Training until validation scores don't improve for 30 rounds
[500]	training's rmse: 2.74769	valid_1's rmse: 2.36255
Early stopping, best iteration is:
[545]	training's rmse: 2.7338	valid_1's rmse: 2.36154


In [224]:
# Permutation importance test
# Create data features and validation data 
features_cols = [col for col in list(grid_df) if col not in remove_features]
valid_df = grid_df[grid_df['d'] > (Last_train - 28)].reset_index(drop=True)

# Make normal prediction with our model and save score
valid_df['preds'] = baseline_model.predict(valid_df[features_cols])
base_score = rmse(valid_df[Target], valid_df['preds'])
print('Standard RMSE', base_score)

Standard RMSE 2.361544573219499


In [225]:
# Looping over all numerical features
for col in features_cols:
    # We will make validation set copy to restore features states on each run
    temp_df = valid_df.copy()
    
    if temp_df[col].dtype.name != "category":
        score_list = []
        for i in range(0, 50):
            temp_df[col] = np.random.permutation(temp_df[col].values)
            temp_df["preds"] = baseline_model.predict(temp_df[features_cols])
            cur_score = rmse(temp_df[Target], temp_df["preds"])
            score_list.append(cur_score)
            #print(score_list)
        
        print(col, np.round(0.02 * sum(score_list) - base_score, 4), 
              np.round(max(score_list) - base_score, 4),
              np.round(min(score_list) - base_score, 4))

release 0.0 0.0 0.0
sell_price 0.0219 0.0312 0.0148
price_min 0.0151 0.0193 0.01
price_max 0.0027 0.0044 0.001
price_mean -0.0001 0.0015 -0.0016
price_std 0.0239 0.0275 0.0198
price_nunique 0.0009 0.0047 -0.0017
item_nunique 0.0193 0.0319 0.0065
month 0.0015 0.003 0.0002
year 0.0 0.0 0.0
prices_momentum 0.0004 0.0024 -0.0014
price_momentum 0.0 0.0004 -0.0005
price_momentum_m 0.0308 0.0438 0.0187
price_momentum_y 0.007 0.0215 0.0005
tm_d 0.0052 0.01 0.0008
tm_w 0.0036 0.0056 0.0007
tm_m 0.0014 0.0032 -0.0012
tm_y 0.0 0.0 0.0
tm_wm -0.0002 0.0009 -0.0008
tm_dw 0.1581 0.1879 0.1302
tm_w_end 0.0149 0.022 0.0043
enc_cat_id_mean 0.0 0.0002 -0.0001
enc_cat_id_std 0.0 0.0 0.0
enc_dept_id_mean 0.0032 0.0042 0.0022
enc_dept_id_std 0.0007 0.0013 0.0001
enc_item_id_mean 0.4706 0.4854 0.4598
enc_item_id_std 0.0468 0.0519 0.0424


In [163]:
# Remove temp data
del temp_df, valid_df